In [2]:
import glob
import pandas as pd
import numpy as np
import os
import subprocess

In [2]:
video_folder = 'VIRAT_Dataset/videos/'
annot_folder = 'VIRAT_Dataset/annotations/'
vid_fn = glob.glob(video_folder + '*.mp4')
vid_event_fn = glob.glob(annot_folder + '*.viratdata.events.txt')
vid_object_fn = glob.glob(annot_folder + '*.viratdata.objects.txt')
len(vid_fn), len(vid_event_fn), len(vid_object_fn)

(329, 275, 315)

In [3]:
# Replacing '\\' in path with '/'. '\\' is the default seperator in windows for python
vid_fn = [path.replace('\\','/') for path in vid_fn]
vid_event_fn = [path.replace('\\','/') for path in vid_event_fn]
vid_object_fn = [path.replace('\\','/') for path in vid_object_fn]
list(zip(vid_fn, vid_event_fn, vid_object_fn))

[('VIRAT_Dataset/videos/VIRAT_S_000001.mp4',
  'VIRAT_Dataset/annotations/VIRAT_S_000001.viratdata.events.txt',
  'VIRAT_Dataset/annotations/VIRAT_S_000001.viratdata.objects.txt'),
 ('VIRAT_Dataset/videos/VIRAT_S_000002.mp4',
  'VIRAT_Dataset/annotations/VIRAT_S_000002.viratdata.events.txt',
  'VIRAT_Dataset/annotations/VIRAT_S_000002.viratdata.objects.txt'),
 ('VIRAT_Dataset/videos/VIRAT_S_000003.mp4',
  'VIRAT_Dataset/annotations/VIRAT_S_000003.viratdata.events.txt',
  'VIRAT_Dataset/annotations/VIRAT_S_000003.viratdata.objects.txt'),
 ('VIRAT_Dataset/videos/VIRAT_S_000004.mp4',
  'VIRAT_Dataset/annotations/VIRAT_S_000004.viratdata.events.txt',
  'VIRAT_Dataset/annotations/VIRAT_S_000004.viratdata.objects.txt'),
 ('VIRAT_Dataset/videos/VIRAT_S_000006.mp4',
  'VIRAT_Dataset/annotations/VIRAT_S_000006.viratdata.events.txt',
  'VIRAT_Dataset/annotations/VIRAT_S_000006.viratdata.objects.txt'),
 ('VIRAT_Dataset/videos/VIRAT_S_000101.mp4',
  'VIRAT_Dataset/annotations/VIRAT_S_000101.viratd

In [4]:
# As evident, the numbers of files are not same. This means few video clips do not have corresponding objects/event file
# We will not use those clips for training which doesn't contain corresponding object files. This means we will use around 315 clips out of 329 clips
# As far as missing event files are concerned, there may be a valid reason for it. The clips may not have any events
# We will create a dataframe containing the columns: video_name, video_path, object_path, event_path
video_df = pd.read_csv('VIRAT_Dataset/docs/list_release2.0.txt', names = ['video_name'])
# defining look-up function
def pathLookup(clip, lst):
    lupath = [path for path in lst if clip in path]
    if len(lupath) !=0:
        return lupath[0]
    else:
        return 'NaN'
    
video_df['video_path'] = video_df.video_name.apply(pathLookup, lst = vid_fn)
video_df['object_path'] = video_df.video_name.apply(pathLookup, lst = vid_object_fn)
video_df['event_path'] = video_df.video_name.apply(pathLookup, lst = vid_event_fn)
video_df['video_name'] = video_df['video_name'] + '.mp4'
# video_df.to_csv('CompleteDataset.csv', index=False)
# video_df[(video_df['object_path'] == 'NaN')].to_csv('VideoMissingObjectFile.csv', index=False)
video_df = video_df[(video_df['object_path'] != 'NaN')]
# video_df.to_csv('FinalDataset.csv', index=False)
video_df

,video_name,video_path,object_path,event_path
0,VIRAT_S_000001.mp4,VIRAT_Dataset/videos/VIRAT_S_000001.mp4,VIRAT_Dataset/annotations/VIRAT_S_000001.virat...,VIRAT_Dataset/annotations/VIRAT_S_000001.virat...
1,VIRAT_S_000002.mp4,VIRAT_Dataset/videos/VIRAT_S_000002.mp4,VIRAT_Dataset/annotations/VIRAT_S_000002.virat...,VIRAT_Dataset/annotations/VIRAT_S_000002.virat...
2,VIRAT_S_000003.mp4,VIRAT_Dataset/videos/VIRAT_S_000003.mp4,VIRAT_Dataset/annotations/VIRAT_S_000003.virat...,VIRAT_Dataset/annotations/VIRAT_S_000003.virat...
3,VIRAT_S_000004.mp4,VIRAT_Dataset/videos/VIRAT_S_000004.mp4,VIRAT_Dataset/annotations/VIRAT_S_000004.virat...,VIRAT_Dataset/annotations/VIRAT_S_000004.virat...
4,VIRAT_S_000006.mp4,VIRAT_Dataset/videos/VIRAT_S_000006.mp4,VIRAT_Dataset/annotations/VIRAT_S_000006.virat...,VIRAT_Dataset/annotations/VIRAT_S_000006.virat...
...,...,...,...,...
324,VIRAT_S_050300_10_002176_002238.mp4,VIRAT_Dataset/videos/VIRAT_S_050300_10_002176_...,VIRAT_Dataset/annotations/VIRAT_S_050300_10_00...,VIRAT_Dataset/annotations/VIRAT_S_050300_10_00...
325,VIRAT_S_050301_00_000000_000036.mp4,VIRAT_Dataset/videos/VIRAT_S_050301_00_000000_...,VIRAT_Dataset/annotations/VIRAT_S_050301_00_00...,VIRAT_Dataset/annotations/VIRAT_S_050301_00_00...
326,VIRAT_S_050301_01_000083_000320.mp4,VIRAT_Dataset/videos/VIRAT_S_050301_01_000083_...,VIRAT_Dataset/annotations/VIRAT_S_050301_01_00...,VIRAT_Dataset/annotations/VIRAT_S_050301_01_00...
327,VIRAT_S_050301_02_000544_000607.mp4,VIRAT_Dataset/videos/VIRAT_S_050301_02_000544_...,VIRAT_Dataset/annotations/VIRAT_S_050301_02_00...,VIRAT_Dataset/annotations/VIRAT_S_050301_02_00...


In [5]:
# Split dataframe for testing and traning (75%-25%)
from sklearn.model_selection import train_test_split
train, test = train_test_split(video_df, test_size=0.25, random_state=42)
# train.to_csv('TrainingData.csv', index=False)
# test.to_csv('TestingData.csv', index=False)
train.shape[0], test.shape[0]

(236, 79)

In [7]:
def videoBB(vid_path, object_path, bbFormat='NotYolo'):
    
    # Generating Objects Dataframe
    objects = pd.read_csv(object_path, sep = '\s', header = None, engine = 'python')
    objects.columns = ['obj_id','obj_dur','frame_num','bb_lt_x','bb_lt_y','bb_width','bb_height','obj_type']
    object_description = {
                                1: 'Person',
                                2: 'Car',
                                3: 'Vehicles',
                                4: 'Object',
                                5: 'Bike/Bicycle'
                        }
    objects['dis_txt'] = objects['obj_id'].astype(str) + ': ' + objects['obj_type'].apply(lambda x: object_description.get(x))
    uniqueObjID = objects['obj_id'].unique().tolist()
    uniqueColID = [(np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255)) for obj in uniqueObjID]
    color_dict = dict(zip(uniqueObjID, uniqueColID))
    objects['color'] = objects['obj_id'].apply(lambda x:  color_dict.get(x))
    
    if bbFormat == 'NotYolo':
        objects['bb_st_co'] = tuple(zip(objects['bb_lt_x'], objects['bb_lt_y']))
        objects['bb_ed_co'] = tuple(zip(objects['bb_lt_x'] + objects['bb_width'], objects['bb_lt_y'] + objects['bb_height']))
        objects['txt_co'] = tuple(zip(objects['bb_lt_x'], objects['bb_lt_y'] - 7))
    
    # Capturing Video File
    import cv2
    
    vidcap = cv2.VideoCapture(vid_path)

    # Window Parameters
    cv2.namedWindow('Video', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Video', 1000, 750)
    
    font = cv2.FONT_ITALIC
#     font = cv2.QT_FONT_BOLD
    frame_count = 0
    
    while(vidcap.isOpened()):

        res, frame = vidcap.read()

        if res == True:                                                   # Check for read response

            # Labelling Objects
            opf = objects[objects.frame_num == frame_count].values.tolist()
            if len(opf) > 0:                                               # If there are non-zero objects for a frame
                for opft in opf:                                           # There can be multiple objects for a frame
                    frame = cv2.putText(frame, opft[8], opft[12], font, 1, opft[9], 2)
                    frame = cv2.rectangle(frame, opft[10], opft[11], opft[9], 3)

            cv2.imshow('Video', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
        frame_count += 1

    vidcap.release()
    cv2.destroyAllWindows()

#     objects = objects[(objects.Obj_Type >= 1) & (objects.Obj_Type <= 5)]
    

videoBB('VIRAT_Dataset/videos/VIRAT_S_000102.mp4', 'VIRAT_Dataset/annotations/VIRAT_S_000102.viratdata.objects.txt')


In [ ]:
def yolo4Inference(vid_path, result_path, weight_path):
    

In [5]:
os.chdir('C:\DS_ML\Retail_Stores_Video_Analytics\YOLOv4\darknet')
yoloCommand = 'darknet.exe detector demo cfg/coco.data cfg/yolov4.cfg C:\DS_ML\Retail_Stores_Video_Analytics\yolov4.weights -ext_output C:\DS_ML\Retail_Stores_Video_Analytics\Test_Video.mp4'

subprocess.run(yoloCommand, shell=True)

CompletedProcess(args='darknet.exe detector demo cfg/coco.data cfg/yolov4.cfg C:\\DS_ML\\Retail_Stores_Video_Analytics\\yolov4.weights -ext_output C:\\DS_ML\\Retail_Stores_Video_Analytics\\Test_Video.mp4', returncode=0)